In [23]:
import pandas as pd
import seaborn as sns
import numpy as np
# from pandasql import sqldf

In [44]:
def preprocessing(df):  

  #drop duplicated columns
  drop_column=['description2', 'Unnamed: 0', 'link', 'author_y', 'title','url', 'title_1']
  df=df.drop(drop_column,axis=1)

  #renaming
  column_names=['author', 'date_recorded', 'views', 'likes', 'title',
       'description_1', 'duration_seg', 'date_released', 'keywords','description_2']
  df.columns=column_names

  #MODIFYING COLUMN: date_recorded
  df['date_recorded']= pd.to_datetime(df['date_recorded'], format='%B %Y')
  
  ##separate data into new Column
  list_months=[]
  list_years=[]
  for i in range(df.shape[0]):
      list_months.append(df['date_recorded'][i].month)
      list_years.append(df['date_recorded'][i].year)
  df['date_recorded_year']=list_years
  df['date_recorded_month']=list_months

  #mMODIFYING COLUMN: date_released
  column='date_released'
  df[column]= pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S')

  ##separate data into new Column
  list_months=[]
  list_years=[]
  list_hours=[]
  list_minutes=[]
  for i in range(df.shape[0]):
      list_months.append(df[column][i].month)
      list_years.append(df[column][i].year)
      list_hours.append(df[column][i].hour)
      list_minutes.append(df[column][i].minute)
  df[column+'_year']=list_years
  df[column+'_month']=list_months
  df[column+'_hour']=list_hours
  df[column+'_minute']=list_minutes

  #MODIFYING COLUMN: 'keywords'
  df_key=df.keywords
  i=0
  df_result=pd.DataFrame()
  ##transforming line into string
  for line in df_key:
    line=(str(line).replace("[","").replace("]","").split(','))
    new_line=[]
    ##removing additional spaces in words and converting the into lower case
    for word in line:
      word=word.lower().replace(' ', '')[1:-1]
      new_line.append(word)
    ##transforming line into string
    new_line=str(new_line).replace("[","").replace("]","")
    ##writting line into dataframe
    df_result.at[i,'keywords2']=new_line
    i=i+1
  
  df=pd.concat([df,df_result], axis=1)

  #drop initial columns
  drop_columns=['date_recorded','date_released', 'keywords']
  df=df.drop(drop_columns, axis=1)

  print(df)

  return(df)

In [42]:
def main():
  #load data
  raw_data=pd.read_csv('https://github.com/aaas24/code_library/raw/main/ted_talks/1_raw_data/final_raw_data.csv')
  df=raw_data.copy()

  #clean data
  df=preprocessing(df)

  return(df)
  


In [45]:
if __name__ == '__main__':
  main()

                    author     views    likes  \
0     Ozawa Bineshi Albert    404000    12000   
1            Sydney Iaukea    214000     6400   
2            Martin Reeves    412000    12000   
3        James K. Thornton    427000    12000   
4          Mahendra Singhi      2400       72   
...                    ...       ...      ...   
5435          Hans Rosling  15000000   458000   
5436      Sir Ken Robinson  72000000  2100000   
5437         Majora Carter   2900000    88000   
5438           David Pogue   2000000    60000   
5439               Al Gore   3600000   109000   

                                                  title  \
0         Climate action needs new frontline leadership   
1           The dark history of the overthrow of Hawaii   
2                    Why play is essential for business   
3     Why is China appointing judges to combat clima...   
4       Cement's carbon problem -- and 2 ways to fix it   
...                                                 ...  

## Data Exploration

In [38]:
df=main()
df.head(5)

,author,views,likes,title,description_1,duration_seg,description_2,date_recorded_year,date_recorded_month,date_released_year,date_released_month,date_released_hour,date_released_minute,keywords2
0,Ozawa Bineshi Albert,404000.0,12000.0,Climate action needs new frontline leadership,"""We can't rely on those who created climate ch...",834.0,"""We can't rely on those who created climate ch...",2021.0,12.0,2022.0,2.0,9.0,41.0,NaN
1,Sydney Iaukea,214000.0,6400.0,The dark history of the overthrow of Hawaii,"""On January 16th, 1895, two men arrived at Lil...",0.0,"""On January 16th, 1895, two men arrived at Lil...",2022.0,2.0,2022.0,2.0,10.0,13.0,NaN
2,Martin Reeves,412000.0,12000.0,Why play is essential for business,"""To thrive in today's competitive economy, you...",665.0,"""To thrive in today's competitive economy, you...",2021.0,9.0,2022.0,2.0,9.0,51.0,NaN
3,James K. Thornton,427000.0,12000.0,Why is China appointing judges to combat clima...,"""Why is China appointing thousands of judges t...",695.0,"""Why is China appointing thousands of judges t...",2021.0,10.0,2022.0,2.0,9.0,45.0,NaN
4,Mahendra Singhi,2400.0,72.0,Cement's carbon problem -- and 2 ways to fix it,"""Cement is vital to modernizing all kinds of i...",671.0,"""Cement is vital to modernizing all kinds of i...",2021.0,10.0,2022.0,2.0,9.0,38.0,NaN


In [9]:
df.dtypes

author                  object
views                    int64
likes                    int64
title                   object
description_1           object
duration_seg             int64
keywords                object
description_2           object
date_recorded_year       int64
date_recorded_month      int64
date_released_year       int64
date_released_month      int64
date_released_hour       int64
date_released_minute     int64
dtype: object

In [10]:
# missing data
((df
 .isna()
 .mean()
 *100)
 .pipe(lambda ser:ser[ser>0])
)

author    0.018382
dtype: float64

### Keywords: 
#### What type of content is published

In [20]:
df_key=df.keywords

print(type(df_key), df_key.shape)


<class 'pandas.core.series.Series'> (5440,)


In [31]:
#initializing
i=0
df_result=pd.DataFrame()
#transforming line into string
for line in df_key:
  line=(str(line).replace("[","").replace("]","").split(','))
  new_line=[]
  #removing additional spaces in words and converting the into lower case
  for word in line:
    word=word.lower().replace(' ', '')[1:-1]
    new_line.append(word)
  #transforming line into string
  new_line=str(new_line).replace("[","").replace("]","")
  #writting line into dataframe
  df_result.at[i,'keywords']=new_line
  i=i+1

#converting into dummy columns
df2=df_result.keywords.str.get_dummies(',')
print(df2.shape)

  


(5440, 349)


In [105]:
df.head(10)

,author,views,likes,title,description_1,duration_seg,keywords,description_2,date_recorded_year,date_recorded_month,date_released_year,date_released_month,date_released_hour,date_released_minute
0,Ozawa Bineshi Albert,404000,12000,Climate action needs new frontline leadership,"""We can't rely on those who created climate ch...",834,"['TED', ' talks', ' climate change', ' global ...","""We can't rely on those who created climate ch...",2021,12,2022,2,9,41
1,Sydney Iaukea,214000,6400,The dark history of the overthrow of Hawaii,"""On January 16th, 1895, two men arrived at Lil...",0,"['TED', ' talks', ' education', ' women', ' TE...","""On January 16th, 1895, two men arrived at Lil...",2022,2,2022,2,10,13
2,Martin Reeves,412000,12000,Why play is essential for business,"""To thrive in today's competitive economy, you...",665,"['TED', ' talks', ' business', ' leadership', ...","""To thrive in today's competitive economy, you...",2021,9,2022,2,9,51
3,James K. Thornton,427000,12000,Why is China appointing judges to combat clima...,"""Why is China appointing thousands of judges t...",695,"['TED', ' talks', ' climate change', ' polluti...","""Why is China appointing thousands of judges t...",2021,10,2022,2,9,45
4,Mahendra Singhi,2400,72,Cement's carbon problem -- and 2 ways to fix it,"""Cement is vital to modernizing all kinds of i...",671,"['TED', ' talks', ' climate change', ' Countdo...","""Cement is vital to modernizing all kinds of i...",2021,10,2022,2,9,38
5,Rosamund Adoo-Kissi-Debrah,422000,12000,The tragedy of air pollution -- and an urgent ...,"""&quot;Breathing clean air is every child's hu...",483,"['TED', ' talks', ' pollution', ' environment'...","""&quot;Breathing clean air is every child's hu...",2021,10,2022,2,9,29
6,Iseult Gillespie,412000,12000,The myth of Narcissus and Echo,"""One day, Echo was drifting through the woods ...",0,"['TED', ' talks', ' education', ' TED-Ed', ' s...","""One day, Echo was drifting through the woods ...",2022,2,2022,2,10,1
7,Gay Gordon-Byrne,455000,13000,You deserve the right to repair your stuff,"'A self-declared ""repair geek,"" Gay Gordon-Byr...",620,"['TED', ' talks', ' sustainability', ' society...","'A self-declared ""repair geek,"" Gay Gordon-Byr...",2021,8,2022,2,9,57
8,Erin Meezan,66000,1900,What nature can teach us about sustainable bus...,"""From glue alternatives inspired by gecko's fe...",235,"['TED', ' talks', ' nature', ' climate change'...","""From glue alternatives inspired by gecko's fe...",2022,2,2022,2,9,53
9,Dwan Reece,584000,17000,The origins of blackface and Black stereotypes,"""If you're wondering why blackface -- mimickin...",0,"['TED', ' talks', ' United States', ' society'...","""If you're wondering why blackface -- mimickin...",2019,3,2022,2,10,5


# Analysis

In [106]:
# #what constitude a good video based on likes?
y=1000
df_graph=df.likes.apply(lambda x: round(x/y,0))
df_graph.describe()

count    5440.000000
mean       62.666912
std       107.730958
min         0.000000
25%        20.000000
50%        41.000000
75%        65.000000
max      2100.000000
Name: likes, dtype: float64

In [107]:
# #what constitude a good video based on views?
# y=1000
# df_graph=df.views.apply(lambda x: round(x/y,0))
# df_graph.describe()

### Top 10 Liked Videos

In [108]:
# #build data: These are the 75% most liked videos
# y_var='likes'
# df_grap = (
#     (df.groupby(['title','author_x','views', 'keywords'])[y_var].sum().reset_index())
#     .sort_values([y_var],ascending=[False])
#     ).reset_index()
# df_grap=df_grap.drop('index', axis=1)
# df_grap = df_grap[df_grap[y_var] > 65000]
# df_grap

In [109]:
# #This is taking the 75% percentile on liked videos
# #Modifiable variables
# y_var='likes'

# #build data
# df_grap = (
#     (df.groupby(['title_2','author','views'])[y_var].sum().reset_index())
#     .sort_values([y_var],ascending=[False])
#     ).reset_index()
# df_grap=df_grap.drop('index', axis=1)
# df_grap = df_grap[df_grap[y_var] > 65000]
# df_grap.head(30)

### Top 10 Authors

In [110]:
# #Modifiable variables
# y_var='views'
# x_var1='author'
# x_var2='likes'

# #build data
# df_grap = (
#     (df.groupby([x_var1, x_var2])[y_var].sum().reset_index())
#     .sort_values([y_var],ascending=[False])
#     ).reset_index().head(10)
# df_grap=df_grap.drop('index', axis=1)
# df_grap

In [111]:
# #visualize
# x_var='author'
# y_var='views'
# sns.set_theme(style="white")
# ax = sns.barplot(x=y_var, y=x_var, data=df_grap, orient="h")

In [112]:
# #visualize
# x_var='author'
# y_var='likes'
# sns.set_theme(style="white")
# ax = sns.barplot(x=y_var, y=x_var, data=df_grap, orient="h", palette="Blues_d")

In [113]:
# q = """
# SELECT
# `title`
# FROM
# df
# LIMIT 5
# """
# print(sqldf(q))

Areas of improvements:
    
    1) More information on the authors. Understanding age, gender and nationality of authors, may answer questions related to diversity of the speakers. This data could be parcially scrapped from Wikipedia as there is a dedicated website that tracks this information. 
    https://en.wikipedia.org/wiki/List_of_TED_speakers